# Extracting images, calculating AWEI index, and pre-processing with the ground truth

Parameters for extraction are in the following cell

In [1]:
import ee
import geemap
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import pyproj
import pandas as pd
import time
import pickle

try:
    ee.Initialize(project='ee-mpozdech')
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def apply_scale_factors(image): # Applies scaling factors for Landstat
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True)

def apply_scale_factors_8(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
        thermal_bands, None, True)


def image_date(image):
    time_start = ee.Date(image.get('system:time_start'))
    return time_start.format('YYYY-MM-dd').getInfo()


Google Earth Engine has initialized successfully!


Setting the parameters for the script.

In [22]:
# Identifier for lake being looked at
lake_name = 'mono'

# Region of Interest to be investigated
#coordinates = [149.35704470513303, -34.98, 149.49550371751621, -35.20614998677668]        # Lake George, NSW
#coordinates = [-118.7987421553789,38.81781714967809,-118.64491594031115,38.57917776421281] # Walker Lake, NV
#coordinates = [-61.543451166162626,-33.65659441108957,-61.382443061249745,-33.7727040846765] # Lake Melincué, FR
coordinates = [-119.15351031680327,38.08285644076688,-118.88698790646308,37.93303372780372]  # Mono Lake, CA


# Parameters
start_date = '1984-03-16'
end_date = '2022-01-01'

max_cloud_cover = 20         #set % of SAT img that can be cloudy

pxl_fill_threshold = 98      #set % of the SAT img that must have data

merge_tolerance = '20 day'   #set how much of a time-difference there can be between SAT and GT images

awei_threshold = 0.85        #set the maximum value of the AWEI index you are willing to accept as water (higher lets more rainclouds pass)

pct_correct_threshold = 40   #set % of the AWEI pixels that must also be classified as water by GT to keep said AWEI image. Lower means more clouds

region = ee.Geometry.Rectangle(coordinates)
tol = pd.Timedelta(merge_tolerance)
current_dir = os.getcwd()
lake_dir = os.path.join(current_dir, "lakes", lake_name)
os.makedirs(lake_dir, exist_ok=True)

Loading the datasets.

In [23]:
# Landstat 5 dataset
sat5_dataset = ( 
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', max_cloud_cover)
)
sat5_dataset = sat5_dataset.map(apply_scale_factors)

sat5_list_num = len(sat5_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT5 collection: ', sat5_list_num)
sat5_list = sat5_dataset.toList(sat5_dataset.size())

# Landstat 7 dataset
sat7_dataset = ( 
    ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', max_cloud_cover)
)
sat7_dataset = sat7_dataset.map(apply_scale_factors)

sat7_list_num = len(sat7_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT7 collection: ', sat7_list_num)
sat7_list = sat7_dataset.toList(sat7_dataset.size())

# Landstat 8 dataset
sat8_dataset = ( 
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', max_cloud_cover)
)
sat8_dataset = sat8_dataset.map(apply_scale_factors_8)

sat8_list_num = len(sat8_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT8 collection: ', sat8_list_num)
sat8_list = sat8_dataset.toList(sat8_dataset.size())

# Ground Truth dataset
gt_dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory') \
    .filterBounds(region) \
    .filterDate(start_date, end_date) #only 1 band: 'water'

gt_list_num = len(gt_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the GT collection: ', gt_list_num)
gt_list = gt_dataset.toList(gt_dataset.size())

Number of images in the SAT5 collection:  371
Number of images in the SAT7 collection:  316
Number of images in the SAT8 collection:  246
Number of images in the GT collection:  453


Loading the GT dataset collection to a dataframe. Date is stored twice to verify the merge later on.

In [24]:
df_gt = pd.DataFrame(columns=['date','gt_img','date_gt'])

for i in range(0,gt_list_num):
    img_id = i
    image = ee.Image(gt_list.get(img_id))
    date = image_date(image)
    
    df_gt.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)
    df_gt.loc[img_id,'gt_img'] = image
    df_gt.loc[img_id,'date_gt'] = date
    
df_gt

,date,gt_img,date_gt
0,1984-04-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-04-01
1,1984-05-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-05-01
2,1984-06-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01
3,1984-07-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01
4,1984-08-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-08-01
...,...,...,...
448,2021-08-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-08-01
449,2021-09-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-09-01
450,2021-10-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01
451,2021-11-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01


Loading the SAT5 dataset collection to a dataframe.

In [25]:
df_sat5 = pd.DataFrame(columns=['date','img','pxl_fill','cloud_cover','landstat'])

for i in range(0,sat5_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat5_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df_sat5.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)

    #compute AWEI index    
    awei = image.expression(
    '4 * (B2 - B5) - (0.25 * B4 + 2.75 * B7)',
    {
        'B2': image.select('SR_B2'), #green
        'B5': image.select('SR_B5'), #SWIR 1
        'B4': image.select('SR_B4'), #NIR
        'B7': image.select('SR_B7')  #SWIR 2
    })
    
    image = image.addBands(awei.rename('awei')) #add calculated index as a band to the SAT image
    df_sat5.loc[img_id,'img'] = image           #save image object to df

    #compute amount of pixels which have some captured data
    band = image.select('SR_B1')    #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df_sat5.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df_sat5.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()
    
    df_sat5.loc[img_id,'landstat'] = 5

df_sat5

,date,img,pxl_fill,cloud_cover,landstat
0,1989-03-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",36.003852,2,5
1,2003-08-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",14.885348,1,5
2,1984-04-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,18,5
3,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5
4,1984-06-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,5
...,...,...,...,...,...
366,2011-08-03 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5
367,2011-08-19 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5
368,2011-09-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5
369,2011-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5


In [6]:
df_sat5

,date,img,pxl_fill,cloud_cover,landstat
0,1986-02-19 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5
1,1986-03-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5
2,1986-09-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,5
3,1986-10-17 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,4,5
4,1986-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,16,5
...,...,...,...,...,...
163,2010-12-06 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,5
164,2010-12-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,5
165,2011-04-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,5
166,2011-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,5


Loading the SAT7 dataset collection to a dataframe.

In [26]:
df_sat7 = pd.DataFrame(columns=['date','img','pxl_fill','cloud_cover','landstat'])

for i in range(0,sat7_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat7_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df_sat7.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)

    #compute AWEI index    
    awei = image.expression(
    '4 * (B2 - B5) - (0.25 * B4 + 2.75 * B7)',
    {
        'B2': image.select('SR_B2'), #
        'B5': image.select('SR_B5'),
        'B4': image.select('SR_B4'),
        'B7': image.select('SR_B7')
    })
    
    image = image.addBands(awei.rename('awei')) #add calculated index as a band to the SAT image
    df_sat7.loc[img_id,'img'] = image           #save image object to df

    #compute amount of pixels which have some captured data
    band = image.select('SR_B1')    #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df_sat7.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df_sat7.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()
    
    df_sat7.loc[img_id,'landstat'] = 7

df_sat7 #35 minutes

,date,img,pxl_fill,cloud_cover,landstat
0,1999-07-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,7
1,1999-07-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,7
2,1999-09-11 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,7
3,1999-09-27 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
4,1999-10-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
...,...,...,...,...,...
311,2021-08-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.757094,0,7
312,2021-09-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.856342,0,7
313,2021-09-23 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.851316,0,7
314,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.997995,2,7


Loading the SAT8 dataset collection to a dataframe.

In [8]:
df_sat7

,date,img,pxl_fill,cloud_cover,landstat
0,1999-07-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
1,1999-08-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
2,1999-11-14 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
3,1999-12-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
4,2000-01-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
...,...,...,...,...,...
233,2021-08-06 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,11,7
234,2021-08-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,7
235,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7
236,2021-10-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,7


In [27]:
df_sat8 = pd.DataFrame(columns=['date','img','pxl_fill','cloud_cover','landstat'])

for i in range(0,sat8_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat8_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df_sat8.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)

    #compute AWEI index    
    awei = image.expression(
    '4 * (B2 - B5) - (0.25 * B4 + 2.75 * B7)',
    {
        'B2': image.select('SR_B3'), #green
        'B5': image.select('SR_B6'), #SWIR 1
        'B4': image.select('SR_B5'), #NIR
        'B7': image.select('SR_B7')  #SWIR 2
    })
    
    image = image.addBands(awei.rename('awei')) #add calculated index as a band to the SAT image
    df_sat8.loc[img_id,'img'] = image           #save image object to df

    #compute amount of pixels which have some captured data
    band = image.select('SR_B1')    #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df_sat8.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df_sat8.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()
    
    df_sat8.loc[img_id,'landstat'] = 8

df_sat8 #+10 mins?

,date,img,pxl_fill,cloud_cover,landstat
0,2013-03-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",50.415961,1.59,8
1,2013-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",7.479222,2.85,8
2,2013-08-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",7.592723,3.83,8
3,2013-08-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",8.688488,2.05,8
4,2013-09-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",8.685482,0.28,8
...,...,...,...,...,...
241,2021-09-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1.42,8
242,2021-10-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999769,2.31,8
243,2021-11-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999306,13.15,8
244,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8.95,8


Merging the dataframes and ordering them by date.

In [28]:
df_sat57 = pd.concat([df_sat5, df_sat7, df_sat8])
df_sat57 = df_sat57.sort_values('date')
df_sat57.reset_index(drop=True,inplace=True)
df_sat57

,date,img,pxl_fill,cloud_cover,landstat
0,1984-04-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,18,5
1,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5
2,1984-06-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,5
3,1984-06-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,5
4,1984-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5
...,...,...,...,...,...
928,2021-11-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.07939,7,7
929,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",7.494281,3.15,8
930,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8.95,8
931,2021-12-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",7.018555,7.36,8


In [11]:
#checkpoint to save and reload data for debugging
#df_sat57.to_pickle('df_sat57_pre_fill_drop.pkl')
#df_sat57 = pd.read_pickle('df_sat57_pre_fill_drop.pkl')
pd.set_option('display.max_rows', 40)
#df_fin
#df_sat57.reset_index(drop=True,inplace=True)
#df_sat57

Dropping all images that had less than the acceptable amount of pixels filled with data.

In [29]:
df_sat57 = df_sat57.drop(df_sat57[df_sat57['pxl_fill'] < pxl_fill_threshold].index) # drops all rows will less than this data
df_sat57.reset_index(drop=True,inplace=True)
df_sat57

,date,img,pxl_fill,cloud_cover,landstat
0,1984-04-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,18,5
1,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5
2,1984-06-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,5
3,1984-06-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,5
4,1984-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5
...,...,...,...,...,...
567,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.997995,2,7
568,2021-11-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999306,13.15,8
569,2021-11-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.07939,7,7
570,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8.95,8


Merging the SAT and GT dataframes based on the date within set tolerance.

In [30]:
df_sat57['date'] = df_sat57['date'].apply(lambda x: x.to_pydatetime()) #converts timestamps to datetime
df_gt['date'] = df_gt['date'].apply(lambda x: x.to_pydatetime())

df_sat_gt = pd.merge_asof(left=df_sat57,right=df_gt,on='date',right_index=False,left_index=False,direction='nearest',tolerance=tol) #merge on dates
df_sat_gt = df_sat_gt.dropna()
df_sat_gt

,date,img,pxl_fill,cloud_cover,landstat,gt_img,date_gt
0,1984-04-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,18,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-05-01
1,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01
2,1984-06-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01
3,1984-06-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01
4,1984-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01
...,...,...,...,...,...,...,...
567,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.997995,2,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01
568,2021-11-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999306,13.15,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01
569,2021-11-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.07939,7,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01
570,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8.95,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-12-01


Reproject the GT band using the SAT projection, then add GT band to the SAT img. Saves statistics about the AWEI and GT images to seperate df for merge.

In [31]:
df_class = pd.DataFrame(columns=['tot_gt_water_pxls','tot_awei_water_pxls','correctly_classified_pxls','pct_correct','max_awei_value'])

for i in range(0,len(df_sat_gt)):
    img = df_sat_gt.loc[i,'img']           #gets SAT img object from df
    img_gt = df_sat_gt.loc[i,'gt_img']     #gets GT img object from df
    sat_proj = img.projection().getInfo()  #gets SAT crs
    img_gt = img_gt.reproject(sat_proj['crs'],sat_proj['transform']) #transforms GT to SAT projection
    img = img.addBands(img_gt.rename('gt'))#add GT as a band to the SAT image
    df_sat_gt.loc[i,'img'] = img           #saves SAT image with GT back to df
    df_sat_gt.loc[i,'gt_img'] = img_gt     #saves reprojected GT image to df
    
    
    ### AWEI WATER COUNT
    awei = img.select('awei')   #selects awei index band from SAT img
    img_awei_water = awei.gt(0) #gets all values > 0 (classified as water in AWEI)
    # counts number of pixels that were marked as water in AWEI
    awei_water_count = img_awei_water.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('awei')
    df_class.loc[i,'tot_awei_water_pxls'] = awei_water_count.getInfo() #save the number of pixels marked as water by AWEI
    
    ### GT WATER COUNT
    water_classified = img_gt.eq(2) #sets values = 2 (classified water in GT) to 1
    # counts number of pixels that were marked as water in GT
    gt_water_count = water_classified.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    df_class.loc[i,'tot_gt_water_pxls'] = gt_water_count.getInfo()
    
    ### CORRECT AWEI COUNT
    compare_awei_gt = water_classified.And(img_awei_water) #gets all pixels that were marked as water by both AWEI and GT
    # counts number of pixels correctly classified as water by AWEI
    correctly_classified_water_pxls = compare_awei_gt.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    df_class.loc[i,'correctly_classified_pxls'] = correctly_classified_water_pxls.getInfo()
    
    # computes % of AWEI pixels that were also classified as water by GT
    if awei_water_count.getInfo() != 0:
        df_class.loc[i,'pct_correct'] = correctly_classified_water_pxls.getInfo() / awei_water_count.getInfo() * 100 #40% of AWEI pxls must also be classified as water by GT
    else:
        df_class.loc[i,'pct_correct'] = 0
        
    # finds the maximum AWEI value found
    max_value = img.reduceRegion(
        reducer=ee.Reducer.max(), 
        geometry=region,
        scale=30
    ).get('awei')
    
    df_class.loc[i,'max_awei_value'] = max_value.getInfo()
        
df_class #+90min

,tot_gt_water_pxls,tot_awei_water_pxls,correctly_classified_pxls,pct_correct,max_awei_value
0,47459,373171.913725,45616,12.223857,3.151051
1,195517,9665,9083,93.978272,0.436389
2,195517,198182,194798,98.292479,2.776013
3,194481,195668,193957,99.12556,0.52454
4,194481,195518,193968,99.207234,0.543116
...,...,...,...,...,...
567,199602,198142.737255,196804,99.324357,0.606875
568,194710,25211.333333,22274,88.349155,0.691836
569,194710,199366.780392,192948,96.780416,1.596517
570,0,204833,0,0.0,1.655512


Merging the stats df with the img df on index.

In [32]:
df_fin = pd.merge(df_sat_gt,df_class,left_index=True,right_index=True,how='inner')
df_fin.reset_index(drop=True,inplace=True)
df_fin

,date,img,pxl_fill,cloud_cover,landstat,gt_img,date_gt,tot_gt_water_pxls,tot_awei_water_pxls,correctly_classified_pxls,pct_correct,max_awei_value
0,1984-04-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,18,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-05-01,47459,373171.913725,45616,12.223857,3.151051
1,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01,195517,9665,9083,93.978272,0.436389
2,1984-06-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01,195517,198182,194798,98.292479,2.776013
3,1984-06-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01,194481,195668,193957,99.12556,0.52454
4,1984-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01,194481,195518,193968,99.207234,0.543116
...,...,...,...,...,...,...,...,...,...,...,...,...
567,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.997995,2,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01,199602,198142.737255,196804,99.324357,0.606875
568,2021-11-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999306,13.15,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01,194710,25211.333333,22274,88.349155,0.691836
569,2021-11-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.07939,7,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01,194710,199366.780392,192948,96.780416,1.596517
570,2021-12-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8.95,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-12-01,0,204833,0,0.0,1.655512


In [19]:
#checkpoint to save and reload data for debugging
df_fin.to_pickle('df_fin_pre_noise_drop.pkl')
#df_fin = pd.read_pickle('df_fin_pre_noise_drop.pkl')
#pd.set_option('display.max_rows', 20)
#print(df_fin)

Dropping rows where the stats showed low correctness in initial AWEI and GT comparison.

In [33]:
df_fin = df_fin.drop(df_fin[(df_fin['pct_correct'] < pct_correct_threshold) & (df_fin['tot_awei_water_pxls'] > 0)].index) # drops all rows with more than 40% of GT pixels not captured and there were AWEI water pixels present
df_fin.reset_index(drop=True,inplace=True)
df_fin = df_fin.drop(df_fin[df_fin['max_awei_value'] > awei_threshold].index) #drop all rows with a max AWEI above the threshold
df_fin.reset_index(drop=True,inplace=True)
df_fin

,date,img,pxl_fill,cloud_cover,landstat,gt_img,date_gt,tot_gt_water_pxls,tot_awei_water_pxls,correctly_classified_pxls,pct_correct,max_awei_value
0,1984-05-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,9,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01,195517,9665,9083,93.978272,0.436389
1,1984-06-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01,194481,195668,193957,99.12556,0.52454
2,1984-07-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,2,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01,194481,195518,193968,99.207234,0.543116
3,1984-08-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-08-01,194180,194606,193562,99.463531,0.559149
4,1985-06-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,4,5,"ee.Image({\n ""functionInvocationValue"": {\n ...",1985-06-01,192568,193788,192113,99.135653,0.555911
...,...,...,...,...,...,...,...,...,...,...,...,...
228,2021-09-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1.42,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-09-01,199650,197623,197267,99.819859,0.453329
229,2021-09-23 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.851316,0,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01,199602,197314,196205,99.437952,0.523701
230,2021-10-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",99.999769,2.31,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01,199602,197977,196847,99.429227,0.4922
231,2021-10-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",98.997995,2,7,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01,199602,198142.737255,196804,99.324357,0.606875


Adding the GT band to the SAT img, and exporting all SAT imgs.

In [ ]:
#print(df_fin['max_awei_value'].max())
#pd.set_option('display.max_rows', 300)
df_fin

In [ ]:
bands_57 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7','awei','gt']
bands_8 = ['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','awei','gt']

for i in range(0,len(df_fin)):
    image = df_fin.loc[i,'img'] #load img from df
    if df_fin.loc[i,'landstat'] == 5 or df_fin.loc[i,'landstat'] == 7: #check if landstat 5/7 or 8 as they have different bands
        image = image.select(bands_57)
    else:
        image = image.select(bands_8)

    capture_date = image_date(image)

    file_loc = os.path.join(lake_dir, capture_date+'.tif') 

    geemap.ee_export_image(image, filename=file_loc, scale=30, region=region, file_per_band=False) #+44mins

In [ ]:
df_fin.to_pickle('df_fin_final.pkl')
#df_fin = pd.read_pickle('df_fin_pre_noise_drop.pkl')

In [ ]:
# Visualizations
vis_rgb = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}

vis_rgb_8 = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_awei = {
    'bands' : ['awei'],
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
    #'opacity' : 0.3
}

vis_gt = {
    'bands': ['gt'],
    'min': 1,
    'max': 2,
    'palette': ['white', 'red'],
    'opacity' : 0.3
}

In [ ]:
#pd.set_option('display.max_rows', 20)
#df_fin

In [ ]:
img_id = 40 #4,17,65

image = df_fin.loc[img_id,'img']
#print(image.bandNames().getInfo())
print(image_date(image))
#image = image.select('awei')

m_mix = geemap.Map()
#m_mix.set_center(149.42, -35.08,11)
m_mix.add_layer(image.clip(region),vis_rgb, 'sat')
#m_mix.add_layer(image.clip(region),vis_awei,'awei')
#m_mix.add_layer(image.clip(region),vis_gt,'gt')
m_mix

In [ ]:
tiff_date = '1999-07-01'

ens_tiff = rasterio.open(lake_dir +'/' + tiff_date +'.tif')
image = ens_tiff.read()

img = np.transpose(image, [1, 2, 0]) # numpy array [row, col, bands]
print("Image tiff array shape",img.shape) #23 bands, 330x573 pxls
img_rgb = img[:,:,[3,2,1]]

plt.imshow(img_rgb)